In [ ]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"
model_name="../model/------llm-models-hf-0503llama-_inst_gousei_lr_5e-6"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_1e-5/checkpoint-47"
model_name="../model/------llm-models-hf-2jaA_inst_gousei_lr_5e-6"
#model_name="../model/------llm-models-hf-1code_inst_gousei_lr_5e-6"

#model_name="../X_merge/mergoo_llama_test"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )
#層の表示
#for name, param in model.named_parameters():
#    print(name)

In [ ]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=200, 
              repetition_penalty=1.2,
              temperature=0.6,
              )


In [ ]:
question="こんにちは"
question="元気ですか?"
question="いい天気ですね"
question="明日の天気は?"
question="計算してください 1+1はいくつですか｡"
question="calculate the folowing quation: 1+1"
question=": 1+1"
#question="いい天気ですね"
#question="フィボナッチ数列を生成するコードを作って"
#question="1+1はいくつですか"
#question="ドラえもんの友達はだれですか?"
#question="レイリー散乱とはなんですか"
question="hello!"
#question="今の天気をjsonで出力: {"

question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
inp=f"{question_template}{question}<SEP>応答<SEP>"

#inp="this is a pen"
#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。指示"
#inp=f"{question_template}{question}応答"
#inp=f"{question}\n応答:"
pipe(inp)[0]["generated_text"][len(inp):]